In [1]:
import os
import pymongo
from bs4 import BeautifulSoup
import lxml
from datetime import date

In [2]:
os.chdir("..")
os.getcwd()

'/Users/toph/Dropbox (EHA)/repositories/pubcrawler'

Here, will:

- Set up a connection to the Mongo `articles` collection
- Define the class Article to load… load from Mongo with a "loads" IF that's how I get the article.

Yeah, so the Article object is passed the `article_dict` from the `articles` collection. If 

In [3]:
articles = pymongo.MongoClient().pmc.articles
articles.find_one()

{'_id': '20_Century_Br_Hist_2015_Sep_27_26(3)_450-476',
 'nxml': '<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">\n<article xmlns:ali="http://www.niso.org/schemas/ali/1.0/" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:mml="http://www.w3.org/1998/Math/MathML" article-type="research-article"><?properties open_access?><front><journal-meta><journal-id journal-id-type="nlm-ta">20 Century Br Hist</journal-id><journal-id journal-id-type="iso-abbrev">20 Century Br Hist</journal-id><journal-id journal-id-type="publisher-id">tweceb</journal-id><journal-id journal-id-type="hwp">tweceb</journal-id><journal-title-group><journal-title>20 Century British History</journal-title></journal-title-group><issn pub-type="ppub">0955-2359</issn><issn pub-type="epub">1477-4674</issn><publisher><publisher-name>Oxford University Press</publisher-name></publisher></journal-meta><article-meta><article-id pub-id-type="pmid"

In [4]:
x = articles.find_one()
type(x)

dict

In [5]:
x["_id"]

'20_Century_Br_Hist_2015_Sep_27_26(3)_450-476'

In [6]:
"""
This class represents an article or publication from PubMed Central. It is designed to load the 
"""

class Article:
    
    def __init__(self, article_dict):
        self._id = article_dict['_id']
        self.nxml = article_dict['nxml']
        self.soup = BeautifulSoup(self.nxml, 'lxml-xml')

    def pub_ids(self):
        pub_ids = {}
        for row in self.soup.front.find_all('article-id'):
            pub_id_type = row['pub-id-type']
            pub_id = row.get_text()
            pub_ids[pub_id_type] = pub_id
        return(pub_ids)

    def pub_dates(self):
        pub_dates = {}
        for row in self.soup.front.find_all('pub-date'):
            pub_type = row['pub-type']
            year = int(row.year.get_text()) if row.year is not None else 1
            month = int(row.month.get_text()) if row.month is not None else 1
            day = int(row.day.get_text()) if row.day is not None else 1
            # pub_date = date(year, month, day)
            pub_dates[pub_type] = date(year, month, day)
        return(pub_dates)
    
    def article_title(self):
        if self.soup.front.find('article-title') is not None:
            article_title = self.soup.front.find('article-title').get_text
        else:
            article_title = None
        return(article_title)

    def abstract(self):
        if self.soup.abstract is not None:
            abstract = self.soup.abstract.get_text()
        else:
            abstract = None
        return(abstract)

    def body(self):
        if self.soup.body is not None:
            body = self.soup.body.get_text()
        else:
            body = None
        return(body)

In [7]:
y = Article(x)

In [8]:
y.body()

'Parents Against Injustice (PAIN) was an advocacy group which, between July 1985 and November 1999, sought to represent parents falsely accused of child abuse. PAIN was founded by Susan and Steve Amphlett after the couple were mistakenly accused of physically injuring their youngest daughter.1 PAIN held the following ‘principal objectives’:\nTo campaign for reform in abuse and care procedures with accepted and enforceable codes of practice.To ensure that the ‘principles of natural justice’ were given ‘due regard’ within these care procedures.To encourage public debate on the rights of parents, children and families involved in child abuse cases.To offer support and advice and to act as a referral helpline to parents unjustly accused of abuse.2\nWith these objectives, the organization served both as a lobbying group, seeking to influence policy-makers, care professionals, and media, and also as a support group, giving falsely accused parents emotional support and advice.3 In both its ca

In [9]:
y.pub_dates()

{'epub': datetime.date(2015, 7, 27), 'ppub': datetime.date(2015, 9, 1)}

In [10]:
y.article_title()

<bound method Tag.get_text of <article-title>Painful Times: The Emergence and Campaigning of Parents Against Injustice in 1980s Britain</article-title>>

In [11]:
print(y.soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">
<article article-type="research-article" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink">
 <?properties open_access?>
 <front>
  <journal-meta>
   <journal-id journal-id-type="nlm-ta">
    20 Century Br Hist
   </journal-id>
   <journal-id journal-id-type="iso-abbrev">
    20 Century Br Hist
   </journal-id>
   <journal-id journal-id-type="publisher-id">
    tweceb
   </journal-id>
   <journal-id journal-id-type="hwp">
    tweceb
   </journal-id>
   <journal-title-group>
    <journal-title>
     20 Century British History
    </journal-title>
   </journal-title-group>
   <issn pub-type="ppub">
    0955-2359
   </issn>
   <issn pub-type="epub">
    1477-4674
   </issn>
   <publisher>
    <publisher-name>
     Oxford Universi

In [12]:
print(y.soup.abstract.prettify())

<abstract>
 <p>
  In July 1985 Steve and Susan Amphlett established Parents Against Injustice (PAIN) to support and represent parents falsely accused of child abuse. The Amphletts ran the organization from their own home, and struggled to gain funding, before closing PAIN in 1999. PAIN was to an extent a reflection of the ‘new politics’ of identity and lifestyle, concurrent with the rise of New Social Movements, as falsely accused parents utilized communication technologies to make their experiences public, and to contact and support one another. At the same time, PAIN also sought to exert political influence through relatively traditional channels—contributing to public inquiries, encouraging their membership to write letters to Members of Parliament, and shaping media critique. Despite its small size, PAIN was able to act as an intermediary between parents and politicians, social workers, solicitors and physicians. PAIN represented, but also collated and shaped, parents’ experiences.

In [13]:
print(y.soup.body.prettify())

<body>
 <p>
  Parents Against Injustice (PAIN) was an advocacy group which, between July 1985 and November 1999, sought to represent parents falsely accused of child abuse. PAIN was founded by Susan and Steve Amphlett after the couple were mistakenly accused of physically injuring their youngest daughter.
  <xref ref-type="fn" rid="hwv024-FN1">
   <sup>
    1
   </sup>
  </xref>
  PAIN held the following ‘principal objectives’:
  <list list-type="order">
   <list-item>
    <p>
     To campaign for reform in abuse and care procedures with accepted and enforceable codes of practice.
    </p>
   </list-item>
   <list-item>
    <p>
     To ensure that the ‘principles of natural justice’ were given ‘due regard’ within these care procedures.
    </p>
   </list-item>
   <list-item>
    <p>
     To encourage public debate on the rights of parents, children and families involved in child abuse cases.
    </p>
   </list-item>
   <list-item>
    <p>
     To offer support and advice and to act as 